### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

### **Objectives**

1. 실습명 : Text-to-SQL task에 대해서 LoRA 학습하기
2. 핵심 주제:
    1. ko_text2sql 데이터셋 불러오기 및 EDA
    2. base 모델 및 토크나이저 불러오기
    3. Text-to-SQL task 데이터 전처리
    4. Text-to-SQL task 모델 LoRA fine-tuning
3. 학습 목표 :
    1. ko_text2sql 데이터셋을 불러오고 EDA를 할 수 있다.
    2. Text-to-SQL task 학습을 위한 base 모델 및 토크나이저를 불러올 수 있다.
    3. ko_text2sql 데이터셋의 전처리 전략을 수립할 수 있다.
    4. base 모델에 LoRA fine-tuning을 진행할 수 있다.
4. 학습 개념: 키워드명 : 주요 학습 키워드에 대한 설명을 1줄로 작성해주세요. 키워드는 3개 이상 작성 부탁드립니다.
    1. EDA
    2. Text-to-SQL
    3. LoRA fine-tuning
5. 학습 방향 :
  - Text-to-SQL이라는 특정 task에 대해서 특화시키는 LoRA fine-tuning을 진행합니다. 모델의 성능이 아쉬울 경우 학습 데이터와 적은 GPU 메모리로도 성능을 효율적으로 높일 수 있는 방법에 대해서 체득합니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하여 학습합니다.
  - 해당 실습은 모델을 학습시킬 경우 무엇이 필요하고 어떻게 하면 학습을 효율적으로 할 수 있는지 고민해봅니다.

6. 데이터셋 개요 및 저작권 정보
  - 데이터셋 명 : [shangrilar/ko_text2sql](https://huggingface.co/datasets/shangrilar/ko_text2sql)
  - 데이터셋 개요 : table 정보와 질문 그리고 SQL문이 포함되어 있는 데이터셋
  - 데이터셋 저작권 : cc-by-4.0



### **Prerequisites**
```
numpy==2.0.2
pandas==2.2.2
transformers==4.56.0
torch==2.8.0+cu126
accelerate==1.10.1
datasets==4.0.0
peft==0.17.1
trl==0.22.2
```

In [1]:
%pip install numpy==2.0.2 pandas==2.2.2 peft==0.17.1 torch==2.8.0+cu126 --index-url https://download.pytorch.org/whl
%pip install transformers==4.56.0 accelerate==1.10.1 datasets==4.0.0
%pip install trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 39.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import numpy as np
import random

# 시드 설정
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 1. 데이터셋 불러오기 및 EDA

- 학습 목표 : ko_text2sql 데이터셋에 대해서 EDA를 할 수 있다.
- 학습 개념 : EDA
- 진행하는 실습 요약
    - 데이터셋 불러오기
    - 데이터셋 EDA


첨부한 데이터셋을 불러옵니다.

In [3]:
from datasets import load_dataset

text_to_sql = load_dataset("shangrilar/ko_text2sql", data_dir="origin")
train_df = text_to_sql["train"].to_pandas()
train_df.head()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/281 [00:00<?, ?B/s]

origin/train.csv:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

,db_id,context,question,answer
0,1,CREATE TABLE players (\n player_id INT PRIMAR...,모든 플레이어 정보를 조회해 줘,SELECT * FROM players;
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,모든 사용자의 아이디와 이메일을 보여줘,"SELECT player_id, email FROM players;"
2,1,CREATE TABLE players (\n player_id INT PRIMAR...,가입한 날짜를 기준으로 모든 회원 정보를 가져와 줘,SELECT * FROM players ORDER BY date_joined;
3,1,CREATE TABLE players (\n player_id INT PRIMAR...,마지막 로그인 시간이 갱신된 모든 사용자를 찾아줘,SELECT * FROM players WHERE last_login IS NOT ...
4,1,CREATE TABLE players (\n player_id INT PRIMAR...,유저 이름을 기준으로 모든 플레이어의 정보를 나열해줘,SELECT * FROM players ORDER BY username;


column과 각 데이터의 타입을 확인합니다.

### ko_text2sql column 설명
db_id, context, question, answer 컬럼 설명

- db_id : DB의 id 정보
- context : DDL의 테이블 정보
- question : 유저의 질문
- answer : 유저의 질문에 따른 SQL문

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38246 entries, 0 to 38245
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   db_id     38246 non-null  int64 
 1   context   38246 non-null  object
 2   question  38246 non-null  object
 3   answer    38246 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [5]:
# 한 줄만 가져오기
for i, row in train_df.iterrows():
    for k, v in row.items():
        print(k, ":", v)
    break

db_id : 1
context : CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);
question : 모든 플레이어 정보를 조회해 줘
answer : SELECT * FROM players;


`db_id` column은 굳이 필요한 컬럼은 아니므로 해당 column을 제거하고 column을 재정의합니다.

In [6]:
columns = ["context", "question", "answer"]

train_data = train_df[columns]
train_data

,context,question,answer
0,CREATE TABLE players (\n player_id INT PRIMAR...,모든 플레이어 정보를 조회해 줘,SELECT * FROM players;
1,CREATE TABLE players (\n player_id INT PRIMAR...,모든 사용자의 아이디와 이메일을 보여줘,"SELECT player_id, email FROM players;"
2,CREATE TABLE players (\n player_id INT PRIMAR...,가입한 날짜를 기준으로 모든 회원 정보를 가져와 줘,SELECT * FROM players ORDER BY date_joined;
3,CREATE TABLE players (\n player_id INT PRIMAR...,마지막 로그인 시간이 갱신된 모든 사용자를 찾아줘,SELECT * FROM players WHERE last_login IS NOT ...
4,CREATE TABLE players (\n player_id INT PRIMAR...,유저 이름을 기준으로 모든 플레이어의 정보를 나열해줘,SELECT * FROM players ORDER BY username;
...,...,...,...
38241,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",최근에 추가된 즐겨찾기 서비스의 제목과 판매자의 이름을 찾아줘.,"SELECT T3.title, T1.full_name FROM users AS T1..."
38242,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",2022년에 등록된 서비스를 북마크한 사용자의 프로필 사진 URL은 무엇인가요?,SELECT u.profile_picture_url FROM users AS u J...
38243,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카테고리 'IT 서비스'에 속한 서비스 중에서 50불 이하의 가격을 가진 서비스는 ...,"SELECT s.title, s.price FROM services AS s JOI..."
38244,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",서비스를 제공하는 사용자들 중에서 판매자 유형의 사용자의 수는 몇 명인가요?,SELECT COUNT(DISTINCT s.seller_id) FROM servic...


모델의 입력으로 들어가는 데이터의 특성을 파악합니다.

NLP에서 중요하게 체크해야 할 EDA 항목은 다음과 같습니다.
1. sequence length
2. language
3. input data domain

위의 사항들을 체크해봅시다.

2번과 3번은 데이터셋의 특성으로 인해 이미 체크가 됩니다.
- language : 한국어 및 SQL 코드
- input data domain : 다양한 일반 표(table)에서 뽑은 데이터(Open domain)

그러면 1번을 직접 코드를 작성하여 체크해봅니다.

In [7]:
# 1. sequence length
# TODO: max_sequence length를 확인할 코드를 작성합니다.
max_sequence = [0, 0, 0]
for i, row in train_data.iterrows():
    for i, (k, v) in enumerate(row.items()):
        max_sequence[i] = max(max_sequence[i], len(str(v)))

max_sequence

[1257, 203, 738]

`max_length`가 모델의 `max_sequence_length`를 넘지 않을 것으로 예상합니다.

<blockquote>
<b>🧠 모델 입력은 token이니까 token 길이를 측정해야 하는거 아닌가요?</b><br>
맞습니다. 그러나 현재 어떤 모델을 사용할지 정해지지 않은 상황에서는 token 길이를 측정하기는 어렵습니다. 따라서, 대략적인 길이를 보고 모델이 결정되면 token 길이를 다시 측정해야 합니다.
</blockquote>

이제부터 모델과 토크나이저를 불러옵시다.


# 2. base 모델 및 토크나이저 불러오기
- 학습 목표 : base 모델과 토크나이저를 불러올 수 있다.
- 학습 개념 : base 모델
- 진행하는 실습 요약
    - base 모델 불러오기
    - 토크나이저 불러오기

access 권한을 받았는지 확인하기 위해 huggingface token이 필요합니다. 아래 huggingface token을 입력해주세요.

In [ ]:
from huggingface_hub import login

hf_token = "hf_허깅키"

login(token=hf_token)

모델과 토크나이저를 불러옵니다.

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B" # 이 모델은 신청해야 사용이 가능함..
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None: # pad_token 설정이 되어있지 않는 경우가 존재합니다.
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

모델의 max_sequence_length를 확인합니다.

max_position_embeddings의 길이는 position embeddings의 길이를 의미합니다.

position embeddings는 입력 토큰에 순서 정보를 넣는 임베딩이므로 해당 차원이 실제 입력 토큰의 길이와 동일합니다.

In [11]:
max_sequence_length = model.config.max_position_embeddings
max_sequence = [0, 0, 0]
for i, row in train_data.iterrows():
    for i, (k, v) in enumerate(row.items()):
        max_sequence[i] = max(max_sequence[i], len(tokenizer(str(v))["input_ids"]))

print("Max sequence length of model:", max_sequence_length)
print("Max sequence length:", max_sequence)

Max sequence length of model: 32768
Max sequence length: [335, 133, 382]


131,072의 길이라면 토크나이징을 한 이후의 `max_length`인 319 + 109 보다 훨씬 크기 때문에 충분히 여유롭습니다.

# 3. Text-to-SQL task 데이터 전처리
- 학습 목표 : chat 형식으로 데이터 전처리를 할 수 있다.
- 학습 개념 : 데이터 전처리
- 진행하는 실습 요약
    - apply_chat_template으로 데이터 전처리하기
    - datasets를 이용하여 Datasets 형식으로 변경하기


허깅페이스 tokenizer에서는 jinja2 포맷의 `chat_template`을 제공합니다.

보통 LLM을 제공하는 회사, 모델마다 `chat_template`이 다릅니다. 또한, 최근에는 `chat_template`을 대부분 지원합니다.

따라서, `chat_template`을 확인하고 `apply_chat_template` 매서드를 이용해서 어떻게 적용이 되는지 확인해봅니다.

In [12]:
if tokenizer.chat_template:
    print("=== chat template 사용 가능 ===\n")
    print(tokenizer.chat_template)
else:
    print("=== chat template 사용 불가능 ===\n")

=== chat template 사용 가능 ===

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou a

Jinja2 포맷이 이상하긴 하지만 코드를 어느정도 이해하시는 분들이라면 이게 어떤 의미인지를 대충 유추해볼 수 있습니다.

```jinja
{% for message in messages %}
```
어떠한 `iterable` 객체를 `for loop`를 사용하는 것을 봐서는 `list` 등의 객체로 넣어줘야 할거 같습니다.


```jinja
{{'<|im_start|>' + message['role'] + '' + message['content'] + '<|im_end|>' + ''}}
```

`message` 내에 보면 `role`과 `content`라는 키값이 필요한 것을 알 수 있습니다. 여기서 `dictionary` 등의 객체를 사용하면 될거 같습니다.

또한, 앞뒤에 `<|im_start|>`, `<|im_end|>`와 같은 스페셜 토큰이 붙는 것을 확인하실 수 있습니다. 이 부분은 저희가 건들 필요 없이 알아서 생성해줍니다.

`apply_chat_template`의 입력으로 넣기 위해서는 chat(conversations) 형식을 지켜서 입력으로 넣어줘야 합니다.

기존에 불러온 데이터프레임을 chat 형식으로 만들고 `apply_chat_template`을 적용해보겠습니다.

In [13]:
# 학습용 데이터 전처리

example = train_df[columns].iloc[0]
messages = [
    {
        "role": "system",
        "content": example["context"] # 여기서는 빠른 확인을 위해 input 열을 넣습니다. 추후에 좀더 system prompt를 고도화할 수 있습니다.
    },
    {
        "role": "user",
        "content": example["question"]
    },
    {
        "role": "assistant",
        "content": example["answer"]
    }
]

print(messages)
print()
chat = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
print(chat)

[{'role': 'system', 'content': 'CREATE TABLE players (\n  player_id INT PRIMARY KEY AUTO_INCREMENT,\n  username VARCHAR(255) UNIQUE NOT NULL,\n  email VARCHAR(255) UNIQUE NOT NULL,\n  password_hash VARCHAR(255) NOT NULL,\n  date_joined DATETIME NOT NULL,\n  last_login DATETIME\n);'}, {'role': 'user', 'content': '모든 플레이어 정보를 조회해 줘'}, {'role': 'assistant', 'content': 'SELECT * FROM players;'}]

<|im_start|>system
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);<|im_end|>
<|im_start|>user
모든 플레이어 정보를 조회해 줘<|im_end|>
<|im_start|>assistant
SELECT * FROM players;<|im_end|>



보시는 바와 같이 모델의 입력에 맞게 형식을 맞춰서 텍스트를 생성해주는 것을 확인하실 수 있습니다.

주의할 점이 크게 2가지가 있습니다.
1. `assistant`는 정답이기 때문에 학습에서만 입력으로 넣어주고 추론에서는 넣어줘서는 안됩니다.
2. `add_generation_prompt=False`는 `assistant`가 들어갔기 때문에 `False`로 설정합니다. 추론용이라면 `True`로 설정해주셔야 합니다.

In [14]:
## 추론용 데이터 전처리
chat_inference = tokenizer.apply_chat_template(messages[:-1], tokenize = False, add_generation_prompt = True)
print(chat_inference)

<|im_start|>system
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);<|im_end|>
<|im_start|>user
모든 플레이어 정보를 조회해 줘<|im_end|>
<|im_start|>assistant



추론용 데이터를 확인해보시면

`SELECT * FROM players;<|im_end|>`

이 사라진 것을 확인하실 수 있습니다.

유의할 점은
1. 모델이 정답으로 생성할 텍스트 : `SELECT * FROM players;`
2. EOS 스페셜 토큰 : `<|im_end|>`

두가지로 구성되어 있는 `assistant` 부분을 학습에 사용하게 됩니다.


이제 모델 입력으로 넣기 위해서 데이터를 전처리하도록 하겠습니다.

데이터프레임에 있는 데이터를 꺼내서 `Dataset` 객체로 변환하는 전처리를 수행합니다.

In [15]:
import datasets

system_prompt = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA."""
user_prompt = """Given the <USER_QUERY> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<SCHEMA>
{context}
</SCHEMA>

<USER_QUERY>
{question}
</USER_QUERY>"""

# TODO: 직접 전처리 코드를 작성해보세요.
def convert_to_conversation(examples, tokenizer):
    train_data = []
    for i in range(len(examples)):
        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt.format(question=examples["question"][i], context=examples["context"][i])
            },
            {
                "role": "assistant",
                "content": examples['answer'][i]
            }
        ]
        train_data.append(
            {
                "messages": messages,
            }
        )
    return train_data

train_data_list = convert_to_conversation(train_data, tokenizer)
train_dataset = datasets.Dataset.from_list(train_data_list)
print(train_dataset["messages"][0])

[{'content': 'You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.', 'role': 'system'}, {'content': "Given the <USER_QUERY> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.\n\n<SCHEMA>\nCREATE TABLE players (\n  player_id INT PRIMARY KEY AUTO_INCREMENT,\n  username VARCHAR(255) UNIQUE NOT NULL,\n  email VARCHAR(255) UNIQUE NOT NULL,\n  password_hash VARCHAR(255) NOT NULL,\n  date_joined DATETIME NOT NULL,\n  last_login DATETIME\n);\n</SCHEMA>\n\n<USER_QUERY>\n모든 플레이어 정보를 조회해 줘\n</USER_QUERY>", 'role': 'user'}, {'content': 'SELECT * FROM players;', 'role': 'assistant'}]


보통 모델을 학습할 때 사용하는 labels는 inputs와 동일합니다.
(모델은 내부적으로 labels를 한 칸 오른쪽으로 shift한 후 Cross-Entropy loss를 계산합니다.
이는 다음 토큰 예측(next token prediction)과 같은 auto-regressive 분류 작업에서 표준 방식입니다.)

따라서 전처리된 샘플은 다음과 같은 형태가 됩니다:
```python
{"input_ids": instruction + model response, "labels": instruction + model response}  # HF 모델이 shift +1 처리를 내부적으로 수행
```

하지만 우리가 하려는 작업은 instruction 부분을 -100으로 대체하는 것입니다:

```python
{"input_ids": instruction + model response, "labels": [-100]*len(instruction) + model response}
```

이렇게 하면 Cross-Entropy 함수에 instruction 토큰은 무시(ignore) 하라고 알려주는 셈입니다.

예를 들어,

```
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.<|im_end|>
<|im_start|>user
Given the <USER_QUERY> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<SCHEMA>
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);
</SCHEMA>

<USER_QUERY>
모든 플레이어 정보를 조회해 줘
</USER_QUERY><|im_end|>
<|im_start|>assistant
SELECT * FROM players;<|im_end|>
```

위에서 학습해야 할 부분(정답 라벨)은
```
SELECT * FROM players;<|im_end|>
```
이 부분이므로, 나머지 부분들은 -100으로 마스킹 처리를 합니다.

In [16]:
def convert_train_data(examples, tokenizer):
    messages = examples["messages"]
    label_message = messages[-1]["content"]
    label_input_ids = tokenizer.encode(
        label_message, add_special_tokens=False, return_tensors="pt"
    ).squeeze(0)

    prompt_input_ids = tokenizer.apply_chat_template(
        messages[:2], add_generation_prompt=False, tokenize=True, return_tensors="pt"
    ).squeeze(0)

    response_start_template_ids = tokenizer.encode("<|im_start|>assistant", return_tensors="pt")[0]

    input_ids = torch.cat(
        [
            prompt_input_ids,
            response_start_template_ids,
            label_input_ids,
            torch.tensor([tokenizer.eos_token_id]),
        ],
        dim=0,
    )
    attention_mask = torch.ones(len(input_ids), dtype=torch.int64)
    labels = torch.cat(
        [
            torch.tensor(
                [-100] * (len(input_ids) - len(label_input_ids) - 1)
            ),  # prompt + label_start_template
            label_input_ids,  # label
            torch.tensor([tokenizer.eos_token_id]),  # [EOS]
        ],
        dim=0,
    )

    return (
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
    )

train_dataset = train_dataset.map(
    lambda x: convert_train_data(x, tokenizer),
    batched=False,
    remove_columns=train_dataset.column_names  # 원본 컬럼 제거하고 새 컬럼만 남김
)

train_dataset

Map:   0%|          | 0/38246 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 38246
})

1. input_ids – 토크나이즈된 시퀀스 데이터
- 토크나이저(tokenizer)가 텍스트를 **서브워드 단위(subword units)**로 분할하고, 이를 **어휘 사전(vocabulary)**에 정의된 **토큰 인덱스(token index)**로 변환합니다.
- 이 시퀀스가 Transformer 모델의 **임베딩 레이어(embedding layer)**에 입력되어, 각 정수가 고차원 벡터로 임베딩됩니다.

2. attention_mask – 시퀀스 마스킹 텐서
- Transformer는 일반적으로 고정된 최대 길이(max sequence length) 입력을 사용합니다.
- 실제 입력 길이가 짧으면 패딩(padding)을 추가해 맞추는데, 이때 패딩 토큰은 self-attention 연산에서 무시되어야 합니다.
- attention_mask는 같은 길이의 바이너리 벡터로,
  - 실제 토큰 위치 → 1
  - 패딩 토큰 위치 → 0
- 이 마스크는 어텐션 스코어(attention score) 계산 시 소프트맥스 이전에 매우 작은 음수(−∞)를 더해, 패딩 위치의 기여도를 완전히 제거합니다.

3. labels – 학습 대상(target) 시퀀스
- labels는 **모델의 출력 로짓(logits)**과 비교할 타겟 시퀀스입니다.
- 언어 모델 학습 시 오토레그레시브(next-token prediction) 방식으로 학습하므로, labels는 보통 input_ids와 동일하지만 손실 계산에서 제외할 토큰 위치를 -100으로 마스킹합니다.
- 이렇게 하면 Cross-Entropy Loss 계산 시 무시된 위치는 loss=0으로 처리됩니다.

In [17]:
print("input_ids : ", train_dataset["input_ids"][0])
print("attention_mask : ", train_dataset["attention_mask"][0])
print("labels : ", train_dataset["labels"][0])

input_ids :  [151644, 8948, 198, 2610, 525, 264, 1467, 311, 7870, 3239, 45488, 13, 14627, 686, 2548, 498, 4755, 304, 6364, 323, 498, 686, 6923, 264, 7870, 3239, 3118, 389, 279, 3897, 7531, 35839, 13, 151645, 198, 151644, 872, 198, 22043, 279, 366, 6448, 31585, 29, 323, 279, 366, 3540, 35839, 8066, 6923, 279, 12159, 7870, 3210, 311, 17179, 279, 12685, 821, 11, 12831, 279, 3239, 594, 19482, 11, 52694, 11, 323, 10802, 16982, 382, 27, 3540, 35839, 397, 22599, 14363, 4217, 2399, 220, 2781, 842, 9221, 37467, 12013, 35340, 58058, 345, 220, 5934, 37589, 7, 17, 20, 20, 8, 72350, 4183, 1770, 345, 220, 2551, 37589, 7, 17, 20, 20, 8, 72350, 4183, 1770, 345, 220, 3552, 8950, 37589, 7, 17, 20, 20, 8, 4183, 1770, 345, 220, 2400, 5374, 73506, 39362, 44618, 4183, 1770, 345, 220, 1537, 13681, 39362, 44618, 198, 317, 522, 3540, 35839, 1339, 27, 6448, 31585, 397, 129439, 81676, 142509, 234, 135152, 31079, 60039, 18411, 97167, 33883, 53989, 246, 198, 522, 6448, 31585, 29, 151645, 198, 151644, 77091, 4858, 

# 4. Text-to-SQL task 모델 LoRA fine-tuning

LoRA 설정과 Trainer 설정을 하고 모델 학습을 진행합니다.

LoRA config 설정을 합니다. 자세한 설정은 [Huggingface LoRA Config](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)를 참고해주세요.

In [18]:
from peft import LoraConfig

# TODO: LoRA Config를 작성하세요.
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]  # safe default for many LLMs
)

### SFTConfig 주요 설정 설명
각 설정들은 기본적인 AI에 대한 지식이 필요한 부분들이 많습니다. 따라서, 각 설정들은 하나하나 블로그 글을 참고해가며 깊이 이해해주세요.

1. output_dir
- 설명: 학습이 끝난 모델 가중치, 체크포인트, 로그를 저장할 디렉터리 (또는 Hugging Face Hub 저장소 이름)
- 중요성: 실험 결과를 재현하거나 이어서 학습하려면 필수로 설정해야 함

2. max_seq_length
- 설명: 한 학습 샘플의 최대 토큰 길이
- 영향:
	1. 길이를 크게 하면 더 긴 문장을 학습할 수 있으나 GPU 메모리 사용량 증가
	2. 너무 작으면 긴 문장이 잘려서 학습 품질 저하

3. packing
	- 설명: 여러 짧은 샘플을 하나의 시퀀스로 이어 붙여(max_seq_length까지) 학습
	- 장점: GPU 메모리 효율 ↑, 학습 속도 ↑ (패딩 낭비 감소)
	- 주의점: 시퀀스 사이를 구분하는 special token(EOS 등)을 넣어야 데이터 섞임 방지
  - PACKING 예시

    데이터셋 샘플이 각각 독립된 시퀀스로 학습되고 학습할 토큰들이 짧으면 패딩이 많아집니다.

    ```
    샘플 1: [USER] 오늘 날씨 어때? [EOS] [PAD] [PAD] [PAD] ...
    샘플 2: [USER] 내일 비 와? [EOS] [PAD] [PAD] [PAD] ...
    샘플 3: [USER] 주말 일정 보여줘 [EOS] [PAD] [PAD] ...
    ```
    GPU는 [PAD]도 연산해야 하므로 메모리 낭비 & 속도 저하가 발생합니다.

    짧은 샘플들을 이어붙이고, 샘플 사이에 EOS 같은 구분 토큰을 추가하여 최대 길이(max_seq_length)에 맞게 하나의 긴 시퀀스로 묶습니다.

    Packed 시퀀스:
    ```
    [USER] 오늘 날씨 어때? [EOS]
    [USER] 내일 비 와? [EOS]
    [USER] 주말 일정 보여줘 [EOS]
    ```
    - 패딩이 거의 없고 GPU 연산 효율 ↑
    - 한 시퀀스 안에서 여러 예시를 한 번에 학습 → 학습 속도 ↑
    - `Packing=False` : `[샘플1][PAD PAD PAD] [샘플2][PAD PAD PAD]`
    - `Packing=True` : `[샘플1][EOS][샘플2][EOS][샘플3][EOS]	`

4. num_train_epochs
- 설명: 전체 데이터셋을 몇 번 반복해서 학습할지 결정
- 팁: LoRA/QLoRA는 적은 Epoch(2~3)으로도 잘 수렴하는 경우가 많음

5. per_device_train_batch_size
- 설명: GPU 한 장(또는 장치 하나)에서 한 번에 처리할 샘플 개수
- 조절 포인트: GPU 메모리에 맞춰 조절. 작게 하면 gradient_accumulation_steps로 보완 가능

6. gradient_accumulation_steps
- 설명: 여러 스텝의 그래디언트를 누적해 한 번만 역전파/가중치 업데이트
- 장점: 작은 배치로도 큰 effective batch size 구현 가능
- 예시: batch_size=1, grad_accum=4 → 실제 batch size = 4와 비슷한 효과
- `gradient_accumulation_steps`은 [링크]()를 참고해주세요.

7. gradient_checkpointing
- 설명: 순전파(forward) 중 일부 중간 값을 저장하지 않고 필요할 때 다시 계산
- 장점: GPU 메모리 절약 (특히 대형 모델 학습 시 필수)
- 단점: 연산량 증가 → 학습 속도 소폭 느려짐

8. optim
- 설명: 사용할 옵티마이저 선택 (adamw_torch_fused는 PyTorch의 fused AdamW)
- 장점: 일반 AdamW보다 빠르고 메모리 효율 좋음 (지원 GPU 필요)

9. logging_steps
- 설명: 몇 step마다 학습 로그를 기록할지
- 팁: 너무 작으면 로그 과다, 너무 크면 학습 추적 힘듦 → 10~50 step 정도 권장

10. save_strategy
- 설명: 모델 체크포인트 저장 시점 (epoch, steps, no)
- 예시: "epoch" → 각 epoch가 끝날 때마다 저장

11. learning_rate
- 설명: 모델 학습률
- 팁: QLoRA 논문에서 2e-4가 안정적이라고 권장
- 주의: 지나치게 높으면 loss 폭주, 낮으면 학습 느림

12. fp16 / bf16
- 설명: 학습 시 연산 정밀도 선택
- fp16: 대부분 GPU 지원, 메모리 절약, 속도 빠름
- bf16: 최신 GPU(Ampere 이상)에서 권장, 안정성 더 좋음
- 팁: Colab A100/T4 → fp16 / A100(Ampere) 이상 → bf16 추천

13. max_grad_norm
- 설명: Gradient clipping 값 (너무 큰 gradient를 잘라 학습 안정화)
- QLoRA 권장값: 0.3

14. warmup_ratio
- 설명: 학습 초기에 learning rate를 천천히 올리는 비율
- 장점: 학습 안정화, 초기 손실 폭주 방지
- QLoRA 논문 값: 0.03 (~3% 단계는 warmup)

15. lr_scheduler_type
- 설명: 학습률 스케줄링 방식
- constant: 학습 내내 일정한 학습률
- (다른 옵션: linear, cosine 등)

16. push_to_hub
- 설명: 학습이 끝난 모델을 Hugging Face Hub에 자동 업로드할지 여부
- 협업/공유: 팀원이나 공개 프로젝트에 유용

17. report_to
- 설명: 학습 메트릭을 어디로 보낼지 (예: "tensorboard", "wandb")
- 장점: 학습 과정 시각화 가능

학습 설정을 진행합니다.

In [19]:
from trl import SFTConfig, SFTTrainer

output_dir = "outputs"
num_train_epochs=1
per_device_train_batch_size = 1 # GPU가 부족하다면 해당 값을 줄여주세요.
gradient_accumulation_steps = 4
warmup_ratio = 0.03
max_steps = 50 # 학습 시간이 오래 걸린다면 해당 값을 줄여주세요.
learning_rate = 5e-5
logging_steps = 1
weight_decay = 0.01
max_grad_norm=1.0
lr_scheduler_type = "linear"
report_to = "none" # Use this for WandB etc
bf16=False
gradient_checkpointing=False
optim="adamw_torch"

train_cfg = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_steps=max_steps,
    max_steps=max_steps,
    fp16=False if bf16 else True,
    bf16=bf16,
    report_to=report_to,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
)

학습을 진행합니다.

In [20]:
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = train_dataset.select(range(max_steps * per_device_train_batch_size)), # max_steps * batch_size 만큼만 학습합니다.
    peft_config=peft_config,
    args = train_cfg,
)

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

In [21]:
trainer_stats = trainer.train()
print(trainer_stats)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
1,1.033500
2,1.003700
3,0.907000
4,0.745200
5,0.293100
6,0.312300
7,0.264900
8,0.227700
9,0.144900
10,0.243600


TrainOutput(global_step=50, training_loss=0.1561551693826914, metrics={'train_runtime': 55.0205, 'train_samples_per_second': 3.635, 'train_steps_per_second': 0.909, 'total_flos': 306627155733504.0, 'train_loss': 0.1561551693826914, 'epoch': 3.88})


In [22]:
# ============================================
# Save LoRA adapter (small)
# ============================================
merge_and_save = False
if merge_and_save:
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"LoRA adapter saved to: {output_dir}")

# ============================================
# (Optional) Merge LoRA into base weights and save a full model
# WARNING: creates a large model; only do this if you need standalone weights
# ============================================
if merge_and_save:
    from peft import AutoPeftModelForCausalLM
    merged_model = AutoPeftModelForCausalLM.from_pretrained(
        output_dir,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
        device_map="auto",
    )
    merged_model = merged_model.merge_and_unload()
    merged_dir = output_dir + "-merged"
    merged_model.save_pretrained(merged_dir, safe_serialization=True)
    tokenizer.save_pretrained(merged_dir)
    print(f"Merged full model saved to: {merged_dir}")


테스트를 진행합니다.

In [23]:
text_idx = len(train_df) - 1

test_input = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt.format(question=train_df["question"][text_idx], context=train_df["context"][text_idx])},
]
print("정답 :", train_df["answer"][text_idx])

inputs = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, return_dict=True, return_tensors="pt")
input_prompt = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, tokenize=False)
with torch.no_grad():
    output_ids = model.generate(
        **inputs.to("cuda"),
        max_new_tokens=128,
        stop_strings=["<|endofturn|>", "<|stop|>"],
        tokenizer=tokenizer
    )

print(tokenizer.batch_decode(output_ids)[0][len(input_prompt) - 1:])

정답 : SELECT s.title, COUNT(b.bookmark_id) AS bookmark_count FROM services AS s JOIN bookmarks AS b ON s.service_id = b.service_id WHERE EXTRACT(YEAR FROM s.created_at) < 2020 GROUP BY s.title ORDER BY bookmark_count DESC LIMIT 1;

SELECT s.title FROM services s JOIN bookmarks b ON s.service_id = b.service_id WHERE s.created_at < '2020-01-01' GROUP BY s.title ORDER BY COUNT(b.bookmark_id) DESC LIMIT 1;<|im_end|>
